**EMNIST Dataset Summary**  
There are six different splits provided in this dataset. A short summary of the dataset is provided below:

* EMNIST ByClass: 814,255 characters. 62 unbalanced classes.
* EMNIST ByMerge: 814,255 characters. 47 unbalanced classes.
* EMNIST Balanced:  131,600 characters. 47 balanced classes.
* EMNIST Letters: 145,600 characters. 26 balanced classes.
* EMNIST Digits: 280,000 characters. 10 balanced classes.
* EMNIST MNIST: 70,000 characters. 10 balanced classes.

[Link](https://www.nist.gov/itl/products-and-services/emnist-dataset)



For this execise we will focus on EMNIST ByClass.

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class EMNISTClassifier(nn.Module):
  """Define the class to create the network."""
  def __init__(self, num_classes=47):
    """Initialise the logic for different layers."""
    super().__init__()
    self.num_classes = num_classes
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=10,kernel_size=3,stride=1,bias=False)#input 28x28x1 output 26x26x10  RF 3x3
    self.conv2 = nn.Conv2d(in_channels=10, out_channels=10,kernel_size=3,stride=1,bias=False)#input 26x26x10 output 24x24x10  RF 5x5
    self.conv3 = nn.Conv2d(in_channels=10, out_channels=20,kernel_size=3,stride=1,bias=False)#input 24x24x10 output 22x22x20  RF 7x7
    self.pool1 = nn.MaxPool2d(2, 2)#input 22x22x20 output 11x11x20  RF 14x14
    self.conv4 = nn.Conv2d(in_channels=20, out_channels=20,kernel_size=3,stride=1,bias=False)#input 11x11x20 output 9x9x20  RF 16x16
    self.conv5 = nn.Conv2d(in_channels=20, out_channels=30,kernel_size=3,stride=1,bias=False)#input 9x9x20 output 7x7x30  RF 18x18
    self.conv6 = nn.Conv2d(in_channels=30, out_channels=num_classes,kernel_size=3,stride=1,bias=False)#input 7x7x30 output 5x5x30  RF 20x20
    self.adpative_pool = nn.AdaptiveAvgPool2d(output_size =(1,1))

  def forward(self, x):
    """Implement feedforward through the network."""
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.conv3(x)
    x = F.relu(x)

    x = self.pool1(x)

    x = self.conv4(x)
    x = F.relu(x)
    x = self.conv5(x)
    x = F.relu(x)

    x = self.conv6(x)
    x = self.adpative_pool(x)
    x = x.view(-1,self.num_classes )
    return F.log_softmax(x)



In [3]:
!pip install torchsummary

In [4]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
number_of_classes =62
model = EMNISTClassifier(number_of_classes).to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
            Conv2d-2           [-1, 10, 24, 24]             900
            Conv2d-3           [-1, 20, 22, 22]           1,800
         MaxPool2d-4           [-1, 20, 11, 11]               0
            Conv2d-5             [-1, 20, 9, 9]           3,600
            Conv2d-6             [-1, 30, 7, 7]           5,400
            Conv2d-7             [-1, 62, 5, 5]          16,740
 AdaptiveAvgPool2d-8             [-1, 62, 1, 1]               0
Total params: 28,530
Trainable params: 28,530
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.22
Params size (MB): 0.11
Estimated Total Size (MB): 0.34
----------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [5]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.EMNIST('../data', train=True, download=True, split = "balanced",
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.EMNIST('../data', train=False,  split = "balanced",
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

Extracting ../data/EMNIST/raw/emnist.zip to ../data/EMNIST/raw
Processing byclass


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Processing bymerge
Processing balanced
Processing letters
Processing digits
Processing mnist
Done!


In [6]:
len(train_loader.dataset), len(test_loader.dataset)

(112800, 18800)

In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    print(f"Epoch:{epoch}")
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} ')
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    print(f"Train accuracy= {correct}/{len(train_loader.dataset)} ({100. * correct / len(train_loader.dataset)})")


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
model = EMNISTClassifier(number_of_classes).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 26):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/882 [00:00<?, ?it/s]

Epoch:1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.591478705406189 batch_id=881 : 100%|██████████| 882/882 [00:25<00:00, 35.10it/s]

Train accuracy= 62812/112800 (55.684397163120565)



  0%|          | 0/882 [00:00<?, ?it/s]

...


  0%|          | 0/882 [00:00<?, ?it/s]


Test set: Average loss: 0.3755, Accuracy: 16407/18800 (87%)

Epoch:25


loss=0.420259952545166 batch_id=881 : 100%|██████████| 882/882 [00:24<00:00, 36.02it/s]

Train accuracy= 100055/112800 (88.70124113475177)



Test set: Average loss: 0.3756, Accuracy: 16339/18800 (87%)

